In [2]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C

In [4]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "1c")
df.methods = df.methods.replace("['decision_tree']", "1c")
df.methods = df.methods.replace("['random_forest']", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
df = df_multi
#df_multi = df[df[(((df.dataset in fsIs_m) | (df.dataset in fsIl_m)) | ((df.dataset in flIs_m) | (df.dataset in flIl_m)))].index]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(360, 4)

In [5]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,360.0,0.754703,0.193643,0.010206,0.734632,0.774774


In [7]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,120,0.722501,0.227257,0.020746,0.681840,0.763163
30,120,0.771920,0.173088,0.015801,0.740951,0.802890
60,120,0.769688,0.173051,0.015797,0.738725,0.800650


In [8]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,216,0.724261,0.210660,0.014334,0.696167,0.752355
3c,72,0.793606,0.160520,0.018918,0.756528,0.830684
fc,72,0.807127,0.149362,0.017602,0.772626,0.841628


In [9]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                       
10          1c       72  0.672502  0.252204  0.029723   0.614246  0.730759
            3c       24  0.797120  0.161778  0.033023   0.732396  0.861845
            fc       24  0.797879  0.158495  0.032353   0.734468  0.861290
30          1c       72  0.752969  0.182217  0.021474   0.710880  0.795059
            3c       24  0.807824  0.153598  0.031353   0.746372  0.869276
            fc       24  0.792869  0.161217  0.032908   0.728369  0.857370
60          1c       72  0.747311  0.182980  0.021564   0.705044  0.789577
            3c       24  0.775874  0.170953  0.034896   0.707479  0.844269
            fc       24  0.830633  0.129912  0.026518   0.778657  0.882608

In [10]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 8, 351) =  2.884, p =  0.0040


In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     2.884
Date:                Fri, 01 Nov 2019   Prob (F-statistic):            0.00401
Time:                        10:18:34   Log-Likelihood:                 92.170
No. Observations:                 360   AIC:                            -166.3
Df Residuals:                     351   BIC:                            -131.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.6725      0.022     30.081      0.000       0.629       0.716
C(time_budget)[T.30]                      0.0805      0.032      2.545      0.011       0.018       0.143
C(time_budget)[T.60]                      0.0748      0.032      2.366      0.019       0.013       0.137
C(methods)[T.3c]                          0.1246      0.045      2.787      0.006       0.037       0.213
C(methods)[T.fc]                          0.1254      0.045      2.804      0.005       0.037       0.213
C(time_budget)[T.30]:C(methods)[T.3c]    -0.0698      0.063     -1.103      0.271      -0.194       0.055
C(time_budget)[T.60]:C(methods)[T.3c]    -0.0961      0.063     -1.519      0.130      -0.220       0.028
C(time_budget)[T.30]:C(methods)[T.fc]    -0.0855      0.063     -1.352      0.177      -0.210       0.039
C(time_budget)[T.60]:C(methods)[T.fc]    -0.0421      0.063     -0.665      0.506      -0.166       0.082
==============================================================================
Omnibus:                       20.074   Durbin-Watson:                   0.666
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.300
Skew:                          -0.512   Prob(JB):                     8.72e-06
Kurtosis:                       3.712   Cond. No.                         11.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.186952,2.0,2.597539,0.075887
C(methods),0.507017,2.0,7.044587,0.001001
C(time_budget):C(methods),0.136412,4.0,0.947668,0.436445
Residual,12.631197,351.0,NaN,NaN


In [14]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 4, 355) =  4.824, p =  0.0008


In [15]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     4.824
Date:                Fri, 01 Nov 2019   Prob (F-statistic):           0.000844
Time:                        10:20:00   Log-Likelihood:                 90.237
No. Observations:                 360   AIC:                            -170.5
Df Residuals:                     355   BIC:                            -151.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.6921      0.019     36.159      0.000       0.654       0.730
C(time_budget)[T.30]     0.0494      0.024      2.019      0.044       0.001       0.098
C(time_budget)[T.60]     0.0472      0.024      1.927      0.055      -0.001       0.095
C(methods)[T.3c]         0.0693      0.026      2.687      0.008       0.019       0.120
C(methods)[T.fc]         0.0829      0.026      3.211      0.001       0.032       0.134
==============================================================================
Omnibus:                       25.023   Durbin-Watson:                   0.662
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.553
Skew:                          -0.562   Prob(JB):                     1.41e-07
Kurtosis:                       3.918   Cond. No.                         3.96
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.186952,2.0,2.599071,0.075757
C(methods),0.507017,2.0,7.048744,0.000995
Residual,12.767609,355.0,NaN,NaN


In [17]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.186952,0.093476,2.0,2.599071,0.075757,0.013888,0.008522
C(methods),0.507017,0.253509,2.0,7.048744,0.000995,0.037664,0.032235
Residual,12.767609,0.035965,355.0,NaN,NaN,NaN,NaN


In [18]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0693 0.0213  0.0083 0.1304   True
    1c     fc   0.0829 0.0043  0.0219 0.1439   True
    3c     fc   0.0135    0.9 -0.0612 0.0882  False
---------------------------------------------------


In [36]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30   0.0193 0.6863 -0.0373 0.0759  False
    10     60   0.0094    0.9 -0.0472  0.066  False
    30     60  -0.0099    0.9 -0.0665 0.0467  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C

In [39]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df = df_binary
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(540, 4)

In [40]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,540.0,0.774845,0.228228,0.009821,0.755552,0.794138


In [41]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,180,0.765301,0.250066,0.018639,0.728769,0.801833
30,180,0.784569,0.219867,0.016388,0.752449,0.816689
60,180,0.774665,0.213975,0.015949,0.743406,0.805925


In [42]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
3c,108,0.804413,0.181960,0.017509,0.770095,0.838731
DT,108,0.771149,0.195583,0.018820,0.734262,0.808036
RF,108,0.793349,0.202158,0.019453,0.755222,0.831476
fc,108,0.813439,0.177456,0.017076,0.779970,0.846907
svm,108,0.691877,0.330111,0.031765,0.629617,0.754136


In [43]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                       
10          3c       36  0.808008  0.186897  0.031149   0.746955  0.869061
            DT       36  0.764803  0.214112  0.035685   0.694860  0.834746
            RF       36  0.764667  0.258164  0.043027   0.680334  0.849001
            fc       36  0.809474  0.189506  0.031584   0.747569  0.871379
            svm      36  0.679553  0.353594  0.058932   0.564046  0.795061
30          3c       36  0.808862  0.171811  0.028635   0.752737  0.864987
            DT       36  0.779112  0.194519  0.032420   0.715569  0.842654
            RF       36  0.814875  0.178671  0.029779   0.756509  0.873241
            fc       36  0.821666  0.168708  0.028118   0.766554  0.876777
            svm      36  0.698332  0.329913  0.054986   0.590560  0.806103
60          3c       36  0.796369  0.191496  0.031916   0.733813  0.858924
            DT       36  0.769532  0.182081  0.030347   0.710052  0.829012
            RF       36  0.800505  0.158088  0.026348   0.748863  0.852147
            fc       36  0.809177  0.178304  0.029717   0.750931  0.867423
            svm      36  0.697745  0.314707  0.052451   0.594940  0.800549

In [44]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 14, 525) =  1.536, p =  0.0939


In [45]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.536
Date:                Fri, 01 Nov 2019   Prob (F-statistic):             0.0939
Time:                        10:10:41   Log-Likelihood:                 42.912
No. Observations:                 540   AIC:                            -55.82
Df Residuals:                     525   BIC:                             8.550
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.8080      0.038     21.389      0.000       0.734       0.882
C(time_budget)[T.30]                       0.0009      0.053      0.016      0.987      -0.104       0.106
C(time_budget)[T.60]                      -0.0116      0.053     -0.218      0.828      -0.117       0.093
C(methods)[T.DT]                          -0.0432      0.053     -0.809      0.419      -0.148       0.062
C(methods)[T.RF]                          -0.0433      0.053     -0.811      0.418      -0.148       0.062
C(methods)[T.fc]                           0.0015      0.053      0.027      0.978      -0.103       0.106
C(methods)[T.svm]                         -0.1285      0.053     -2.404      0.017      -0.233      -0.024
C(time_budget)[T.30]:C(methods)[T.DT]      0.0135      0.076      0.178      0.859      -0.135       0.162
C(time_budget)[T.60]:C(methods)[T.DT]      0.0164      0.076      0.217      0.829      -0.132       0.165
C(time_budget)[T.30]:C(methods)[T.RF]      0.0494      0.076      0.653      0.514      -0.099       0.198
C(time_budget)[T.60]:C(methods)[T.RF]      0.0475      0.076      0.628      0.530      -0.101       0.196
C(time_budget)[T.30]:C(methods)[T.fc]      0.0113      0.076      0.150      0.881      -0.137       0.160
C(time_budget)[T.60]:C(methods)[T.fc]      0.0113      0.076      0.150      0.881      -0.137       0.160
C(time_budget)[T.30]:C(methods)[T.svm]     0.0179      0.076      0.237      0.813      -0.130       0.166
C(time_budget)[T.60]:C(methods)[T.svm]     0.0298      0.076      0.395      0.693      -0.119       0.178
==============================================================================
Omnibus:                      142.969   Durbin-Watson:                   0.286
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              273.780
Skew:                          -1.521   Prob(JB):                     3.54e-60
Kurtosis:                       4.708   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.033422,2.0,0.325284,0.722468
C(methods),1.037183,4.0,5.047296,0.000535
C(time_budget):C(methods),0.033915,8.0,0.082521,0.999612
Residual,26.970938,525.0,NaN,NaN


In [47]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 6, 533) =  3.522, p =  0.0020


In [48]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     3.522
Date:                Fri, 01 Nov 2019   Prob (F-statistic):            0.00199
Time:                        10:10:48   Log-Likelihood:                 42.572
No. Observations:                 540   AIC:                            -71.14
Df Residuals:                     533   BIC:                            -41.10
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7949      0.026     31.016      0.000       0.745       0.845
C(time_budget)[T.30]     0.0193      0.024      0.812      0.417      -0.027       0.066
C(time_budget)[T.60]     0.0094      0.024      0.395      0.693      -0.037       0.056
C(methods)[T.DT]        -0.0333      0.031     -1.086      0.278      -0.093       0.027
C(methods)[T.RF]        -0.0111      0.031     -0.361      0.718      -0.071       0.049
C(methods)[T.fc]         0.0090      0.031      0.295      0.768      -0.051       0.069
C(methods)[T.svm]       -0.1125      0.031     -3.674      0.000      -0.173      -0.052
==============================================================================
Omnibus:                      144.638   Durbin-Watson:                   0.286
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              279.680
Skew:                          -1.531   Prob(JB):                     1.86e-61
Kurtosis:                       4.749   Cond. No.                         6.45
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.033422,2.0,0.329826,0.719195
C(methods),1.037183,4.0,5.117771,0.000472
Residual,27.004853,533.0,NaN,NaN


In [50]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.033422,0.016711,2.0,0.329826,0.719195,0.001190,-0.002414
C(methods),1.037183,0.259296,4.0,5.117771,0.000472,0.036943,0.029671
Residual,27.004853,0.050666,533.0,NaN,NaN,NaN,NaN


In [51]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     DT  -0.0333 0.7882  -0.117  0.0505  False
    3c     RF  -0.0111    0.9 -0.0948  0.0727  False
    3c     fc    0.009    0.9 -0.0747  0.0928  False
    3c    svm  -0.1125 0.0024 -0.1963 -0.0288   True
    DT     RF   0.0222    0.9 -0.0615  0.1059  False
    DT     fc   0.0423 0.6211 -0.0414   0.126  False
    DT    svm  -0.0793 0.0736  -0.163  0.0045  False
    RF     fc   0.0201    0.9 -0.0636  0.1038  False
    RF    svm  -0.1015 0.0086 -0.1852 -0.0177   True
    fc    svm  -0.1216  0.001 -0.2053 -0.0378   True
----------------------------------------------------
